# Scraping Yahoo Finance

## Week 7. Practice Programming Assignment 1

In this assignment you are required to look at historical data for 30 companies from [Dow Jones Index](https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average). Tickers for companies from the index can be found in *dow_jones_tickers.txt*. For each company you should get historical daily stock prices for 2019 from https://finance.yahoo.com/, and then use the data to answer the questions you will find below. 

###### Import coursera grader tools

In [1]:
import sys
sys.path.append("..")
import grading
grader = grading.Grader(assignment_key="FpZrXMbETcuStX7z6jFv2Q", 
                      all_parts=["RbP2k", "LLRai", "hgIbw", "Tx3OG", "Y05pG",
                                 "lFPeF", "htUtf", "Xri0I", "4JfUm", "oD7pP"])

In [162]:
# token expires every 30 min
COURSERA_EMAIL = 'susanmalkin@gmail.com'
COURSERA_TOKEN = 'GZF96IdkFBEaIKgM'

<br><br><br><br>

### Coding part

In [3]:
# you can start your web-scraping here

In [4]:
dow_jones_companies = []

with open('dow_jones_tickers.txt') as f:
    for ticker in f:
        dow_jones_companies.append(ticker.strip())

In [5]:
print(dow_jones_companies)

['AXP', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'XOM', 'GS', 'HD', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PFE', 'PG', 'TRV', 'UNH', 'RTX', 'VZ', 'V', 'WBA', 'WMT', 'DIS', 'DOW']


In [16]:
import time
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import datetime
import pandas as pd

In [7]:
data = []

In [10]:

dow_jones_companies = ['BA', 'MCD', 'UNH', 'WBA', 'DIS']

In [11]:
for company in dow_jones_companies:
    
    url = 'https://finance.yahoo.com/quote/' + company + '/history?p=' + company
    driver = webdriver.Firefox()
    driver.get(url)
    driver.implicitly_wait(5)
    
    time_period = driver.find_elements_by_xpath("//span[starts-with(text(), 'Time Period')]")
    arrow_button = driver.find_element_by_xpath("//*[local-name()='svg' and @data-icon='CoreArrowDown']")
    arrow_button.click()
    driver.implicitly_wait(5)
    
    if company == 'DOW':
        
        datefield = driver.find_element_by_xpath("//input[@name='startDate'][@type='date']")
        datefield.clear()
        ActionChains(driver).move_to_element(datefield).perform()
        ActionChains(driver).send_keys_to_element(datefield, '03212019').perform()

        datefield = driver.find_element_by_xpath("//input[@name='endDate'][@type='date']")
        datefield.clear()
        ActionChains(driver).move_to_element(datefield).perform()
        ActionChains(driver).send_keys_to_element(datefield, '01012020').perform()
    
    else:
    
        datefield = driver.find_element_by_xpath("//input[@name='startDate'][@type='date']")
        datefield.clear()
        ActionChains(driver).move_to_element(datefield).perform()
        ActionChains(driver).send_keys_to_element(datefield, '01012019').perform()

        datefield = driver.find_element_by_xpath("//input[@name='endDate'][@type='date']")
        datefield.clear()
        ActionChains(driver).move_to_element(datefield).perform()
        ActionChains(driver).send_keys_to_element(datefield, '01012020').perform()

    driver.find_element_by_xpath("//span[text()='Done']").click()
    driver.implicitly_wait(3)
    driver.find_element_by_xpath("//span[text()='Apply']").click()
    time.sleep(5)
    
    
    i = 0
    row_count = 0
    
    while True:
            
        # driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")   
        driver.find_element_by_tag_name('html').send_keys([Keys.PAGE_DOWN for i in range(10)])
        time.sleep(3)
        i += 1
        if i > 3:
            for rows in driver.find_elements_by_xpath("//tr"):

                if '2019' in rows.text and 'Dividend' not in rows.text:
                    record = []
                    record.append(company)
                    row_count += 1
                    record.append(datetime.datetime.strptime(rows.text[0:12], '%b %d, %Y'))
                    for d in rows.text[12:].split():
                        try:
                            record.append(float(d))
                        except:
                            record.append(int(d.replace(',','')))
                        
                    data.append(record)
            break
    
    driver.quit()
    print(company, row_count)

BA 252
MCD 252
UNH 252
WBA 252
DIS 252


In [12]:
print(len(data))


7506


In [13]:
data.append(['DOW', datetime.datetime(2019, 3, 20, 0, 0),52.75,53.50,49.50,49.79,43.39,2350800])

In [19]:
df = pd.DataFrame(data)

In [32]:
df.rename(columns={0: 'company', 1: 'date', 2: 'open', 3: 'high', 4: 'low', 5: 'close', 6: 'adj_close', 7:'volume' }, inplace=True)
year_end = df[df['date'] == '2019-12-31']
year_start = df[df['date'] == '2019-01-02']

29

In [59]:
ann_prices = year_end.merge(year_start, on='company', how='outer')
ann_prices.loc[24, 'open_y'] = 52.75
ann_prices['ann_percent'] = ((ann_prices['close_x']/ann_prices['open_y'])-1)*100
ann_prices['ann_percent'].mean()

23.665016202328882

In [83]:
df.groupby('company')['volume'].sum().sort_values()

company
TRV       327765900
GS        649800700
MMM       692802900
AXP       822191300
MCD       835862100
DOW       936749000
IBM       990437937
CAT      1028899700
HD       1056470300
UNH      1096118600
RTX      1350333644
BA       1354462300
WBA      1442372900
WMT      1522557600
CVX      1592796900
NKE      1613599100
JNJ      1786792000
V        1904911500
PG       1907446100
MRK      2574389785
DIS      2574885500
XOM      2928142100
JPM      2965728900
VZ       3230374100
KO       3236708800
CSCO     5172391400
INTC     5519974300
PFE      6053914076
MSFT     6194410500
AAPL    28254942800
Name: volume, dtype: int64

In [91]:
df['daily_change'] = ((df['close']/df['open'])-1)*100
df.sort_values('daily_change')

,company,date,open,high,low,close,adj_close,volume,daily_change
6929,UNH,2019-04-16,238.01,238.01,215.82,220.96,212.30,27361400,-7.163565
6296,BA,2019-10-18,369.00,369.35,344.00,344.00,339.98,13544300,-6.775068
7213,WBA,2019-03-01,70.85,71.18,66.08,66.61,59.93,10298900,-5.984474
6931,UNH,2019-04-12,236.50,236.78,220.77,223.22,214.47,13979400,-5.615222
7506,DOW,2019-03-20,52.75,53.50,49.50,49.79,43.39,2350800,-5.611374
...,...,...,...,...,...,...,...,...,...
1753,GS,2019-01-16,187.00,198.15,185.60,197.08,186.28,15194200,5.390374
6916,UNH,2019-05-06,228.17,240.57,227.50,240.50,231.07,7180200,5.403866
7081,WBA,2019-09-09,52.65,55.70,52.65,55.59,50.90,9416400,5.584046
7286,DIS,2019-11-13,138.58,149.92,136.84,148.72,147.83,45725400,7.317073


In [134]:
df['month'] = pd.DatetimeIndex(df['date']).month
df

,company,date,open,high,low,close,adj_close,volume,daily_change,month
0,AXP,2019-12-31,124.29,124.57,123.78,124.49,120.90,2340400,0.160914,12
1,AXP,2019-12-30,125.20,125.46,124.18,124.30,120.71,2306500,-0.718850,12
2,AXP,2019-12-27,125.84,125.97,125.11,125.19,121.58,1788600,-0.516529,12
3,AXP,2019-12-26,124.98,125.44,124.53,125.41,121.79,1486600,0.344055,12
4,AXP,2019-12-24,124.95,125.33,124.38,124.74,121.14,953500,-0.168067,12
...,...,...,...,...,...,...,...,...,...,...
7502,DIS,2019-01-07,109.91,111.40,109.30,110.56,109.23,6714700,0.591393,1
7503,DIS,2019-01-04,107.94,110.75,107.25,109.61,108.29,10122800,1.547156,1
7504,DIS,2019-01-03,108.48,108.65,105.94,106.33,105.05,10594700,-1.981932,1
7505,DIS,2019-01-02,108.10,109.14,107.73,108.97,107.65,9723500,0.804810,1


In [158]:
monthly_min = df.groupby(['company', 'month'])['date'].min()
monthly_max = df.groupby(['company', 'month'])['date'].max()
#monthly = monthly_min.to_frame().merge(monthly_max, on=['company','month'], how='inner')
#monthly

a = monthly_min.to_frame().merge(df, on=['company','date'], how='inner')
b = monthly_max.to_frame().merge(df, on=['company','date'], how='inner')
c = a.merge(b, on=['company', 'month'], how='inner')
c['mon_per'] = ((c['close_y']/c['open_x'])-1)*100
c.groupby(['company','month'])['mon_per'].min().sort_values()

company  month
DOW      5       -17.705033
AAPL     5       -16.580903
CSCO     8       -16.171203
WBA      4       -15.704170
MMM      5       -15.694760
                    ...    
DIS      11       15.718757
IBM      1        20.013074
GS       1        20.495345
BA       1        21.958316
DIS      4        22.743973
Name: mon_per, Length: 358, dtype: float64

<br><br>

### Questions

<br><br>

**Question 1.** What is the average change of price over the year (in %)?

*Note 1*. The opening price is the price at which a stock first trades upon the opening of an exchange on a trading day.

*Note 2*. The closing price for any stock is the final price at which it trades during regular market hours on any given day.

*Note 3*. Here by the price change we going to mean a ratio of a closing price in the last day of the period to an opening price in the first day of that period, subtracted one and multiplied by 100.

Example: if a price of a stock in day 1 opened at \\$100, and its close price in the last day was \\$120, then the price change during the period is: $$\left(\dfrac{120}{100}-1\right) * 100 = (1.2 - 1) * 100=20.$$

The price grew by 20%.

In [163]:
answer_part_1 = 23.665016202328882

In [164]:
# Setting our answers to grader. Do not change!
grader.set_answer("RbP2k", answer_part_1)

<br>

**Question 2.** What company's stock price grew the most (in %)? Enter ticker of the company as an answer)

In [165]:
answer_part_2 = 'AAPL'

In [166]:
# Setting our answers to grader. Do not change!
grader.set_answer("LLRai", answer_part_2)

<br>

**Question 3.** What company's stock lost in price the most (in %)? Enter ticker of the company as an answer

In [167]:
answer_part_3 = 'WBA'

In [168]:
# Setting our answers to grader. Do not change!
grader.set_answer("hgIbw", answer_part_3)

<br>

**Question 4.** What company had the largest summary volume over the year? Enter ticker of the company as an answer

In [169]:
answer_part_4 = 'AAPL'

In [170]:
# Setting our answers to grader. Do not change!
grader.set_answer("Tx3OG", answer_part_4)

<br>

**Question 5.** What is the biggest stock price daily increase (in %)? Enter the number 

In [171]:
answer_part_5 = 7.740997

In [172]:
# Setting our answers to grader. Do not change!
grader.set_answer("Y05pG", answer_part_5)

<br><br>

**Question 6.** What is the company that had the biggest stock price daily increase? Enter ticker of the company as an answer

In [173]:
answer_part_6 = 'BA'

In [174]:
# Setting our answers to grader. Do not change!
grader.set_answer("lFPeF", answer_part_6)

<br>

**Question 7.** What is the biggest stock price daily decrease (in %)? Enter the number

In [175]:
answer_part_7 = -7.163565

In [176]:
# Setting our answers to grader. Do not change!
grader.set_answer("htUtf", answer_part_7)

<br>

**Question 8.** What is the company that had the biggest stock price daily decrese (in %)? Enter ticker of the company as an answer

In [177]:
answer_part_8 = 'UNH'

In [178]:
# Setting our answers to grader. Do not change!
grader.set_answer("Xri0I", answer_part_8)

<br>

**Question 9.** What was the best month for all companies (i.e. average monthly price increase was the best)? Enter one of the following: January, February, March, April, May, June, July, August, September, October, November, December

In [179]:
answer_part_9 = 'January'

In [180]:
# Setting our answers to grader. Do not change!
grader.set_answer("4JfUm", answer_part_9)

<br>

**Question 10.** What was the worst month for all companies (i.e. average monthly price increase was the worst)? Enter one of the following: January, February, March, April, May, June, July, August, September, October, November, December

In [181]:
answer_part_10 = 'May'

In [182]:
# Setting our answers to grader. Do not change!
grader.set_answer("oD7pP", answer_part_10)

<br>
<br>

### Submitting answers

In [183]:
# you can make submission with answers so far to check yourself at this stage
grader.submit(COURSERA_EMAIL, COURSERA_TOKEN)

Submitted to Coursera platform. See results on assignment page!
